In [ ]:
!pip install torchvision


     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------- ----------------------------- 0.3/1.1 MB 6.3 MB/s eta 0:00:01
     ------------------------------------- -- 1.1/1.1 MB 13.7 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 12.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/192.2 MB ? eta -:--:--
     --------------------------------------- 1.4/192.2 MB 29.0 MB/s eta 0:00:07
     --------------------------------------- 2.4/192.2 MB 30.5 MB/s eta 0:00:07
      -------------------------------------- 3.4/192.2 MB 27.1 MB/s eta 0:00:07
      -------------------------------------- 4.5/192.2 MB 28.7 MB/s eta 0:00:07
     - ------------------------------------- 5.5/192.2 MB 25.2 MB/s eta 0:00:08
     - ------------------------------------- 7.0/192.2 MB 28.1 MB/s eta 0:00:

In [1]:
#import necessary libraries

import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms


In [ ]:
'''
import tensorflow as tf
from tensorflow.python.client import device_lib

# Check GPU availability and set the device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
print("PyTorch Device:", device)

# Check TensorFlow version
tf_version = tf.__version__
print("TensorFlow version:", tf_version)

# Print GPU information from TensorFlow
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

# Set PyTorch device to GPU if available
torch_device = torch.device(device)
print("PyTorch Device (after setting):", torch_device)

# Now, you can use the PyTorch device for operations
tensor_on_gpu = torch.randn(3, 3).to(torch_device)
tensor_on_gpu


# Move a PyTorch model to the GPU
#model.to(torch_device)
'''

PyTorch Device: cpu
TensorFlow version: 2.5.0
Found GPU at: /device:GPU:0
PyTorch Device (after setting): cpu


tensor([[ 0.4370, -0.5966,  0.6961],
        [ 0.5541, -0.7279,  0.0381],
        [-0.2652,  0.8922, -1.0369]])

Load pre-trained ViT model

In [43]:
# 1. Get pretrained weights for ViT-Base
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

# 2. Setup a ViT model instance with pretrained weights
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights)#.to(device)


# 3. Freeze the base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

# 4. Change the classifier head
class_names = ['Billboard', 'No-billboard']  #Billboard'--->tensor(0), 'No-billboard'---> tensor(1)


#additional layers
additional_layers = nn.Sequential(
    #nn.Dropout(0.5),
    nn.Linear(in_features=1000, out_features=768),  # Another example linear layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Dropout(0.2),
)

# 5. Combine the pretrained ViT and additional layers
pretrained_vit_addtn = nn.Sequential(
    pretrained_vit,
    additional_layers,
)

# 6. Change the classifier head
class_names = ['Billboard', 'No-billboard']
pretrained_vit_addtn.heads = nn.Linear(in_features=768, out_features=len(class_names))
print(pretrained_vit_addtn)

#set_seeds()
#pretrained_vit.heads = nn.Linear(in_features=768, out_features=len(class_names))#.to(device)
#print(pretrained_vit)


Sequential(
  (0): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): LayerNor

In [ ]:
!pip install torchinfo

In [44]:
from torchinfo import summary

# Print a summary using torchinfo (uncomment for actual output)
summary(model=  pretrained_vit_addtn, #pretrained_vit,#
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                           Input Shape          Output Shape         Param #              Trainable
Sequential (Sequential)                                           [32, 3, 224, 224]    [32, 2]              --                   Partial
├─VisionTransformer (0)                                           [32, 3, 224, 224]    [32, 1000]           768                  False
│    └─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 768, 14, 14]    (590,592)            False
│    └─Encoder (encoder)                                          [32, 197, 768]       [32, 197, 768]       151,296              False
│    │    └─Dropout (dropout)                                     [32, 197, 768]       [32, 197, 768]       --                   --
│    │    └─Sequential (layers)                                   [32, 197, 768]       [32, 197, 768]       (85,054,464)         False
│    │    └─LayerNorm (ln)                          

In [4]:
# Get automatic transforms from pretrained ViT weights
pretrained_vit_transforms = pretrained_vit_weights.transforms()
print(pretrained_vit_transforms)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


Load the dataset

In [5]:
# Setup directory paths to train and test images
train_dir = '/python_programming/FYP_2022/billboard_dataset/train'
val_dir =  '/python_programming/FYP_2022/billboard_dataset/val'
test_dir = '/python_programming/FYP_2022/billboard_dataset/test'


In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def create_dataloaders(
    train_dir: str,
    val_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
):

  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  val_data = datasets.ImageFolder(val_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  print(train_data)
  # Get class names
  class_names = train_data.classes
  print(val_data)

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      pin_memory=True,
  )
  val_dataloader = DataLoader(
      val_data,
      batch_size=batch_size,
      shuffle=False,
      pin_memory=True,
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      pin_memory=True,
  )

  return train_dataloader, val_dataloader, test_dataloader, class_names

In [7]:
# Setup dataloaders
train_dataloader_pretrained, val_dataloader_pretrained, test_dataloader_pretrained,  class_names = create_dataloaders(train_dir=train_dir,
                                                                                                     val_dir = val_dir,
                                                                                                     test_dir = test_dir,
                                                                                                     transform=pretrained_vit_transforms,
                                                                                                     batch_size=16)#32

train_dataloader_pretrained, val_dataloader_pretrained, test_dataloader_pretrained, class_names

Dataset ImageFolder
    Number of datapoints: 2000
    Root location: /python_programming/FYP_2022/billboard_dataset/train
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BILINEAR
           )
Dataset ImageFolder
    Number of datapoints: 1000
    Root location: /python_programming/FYP_2022/billboard_dataset/val
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BILINEAR
           )


(<torch.utils.data.dataloader.DataLoader at 0x232ee9bb0a0>,
 ['Billboard', 'No_Billboard'])

In [8]:

# Get a batch of images
image_batch, label_batch = next(iter(train_dataloader_pretrained))

# Get a single image from the batch
image, label = image_batch[0], label_batch[0]


# View the batch shapes
print(image.shape, label)
print('label:', class_names[label])


#plt.imshow(train_dataloader_pretrained[image])

'''
# Plot image with matplotlib
plt.imshow(image.permute(1, 2, 0)) # rearrange image dimensions to suit matplotlib [color_channels, height, width] -> [height, width, color_channels]
plt.title(class_names[label])
plt.axis(False);
'''

torch.Size([3, 224, 224]) tensor(0)
label: Billboard


'\n# Plot image with matplotlib\nplt.imshow(image.permute(1, 2, 0)) # rearrange image dimensions to suit matplotlib [color_channels, height, width] -> [height, width, color_channels]\nplt.title(class_names[label])\nplt.axis(False);\n'

Train the model

In [9]:
import csv
import h5py

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer) -> Tuple[float, float]:

    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Variables for precision, recall, and F1 score
    train_TP, train_FP, train_FN = 0, 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X, y #X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

        # Update TP, FP, FN for precision, recall, and F1 score
        train_TP += ((y_pred_class == y) & (y == 1)).sum().item()
        train_FP += ((y_pred_class != y) & (y == 0)).sum().item()
        train_FN += ((y_pred_class != y) & (y == 1)).sum().item()

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    # Calculate precision, recall, and F1 score
    train_precision = train_TP / (train_TP + train_FP + 1e-12)
    train_recall = train_TP / (train_TP + train_FN + 1e-12)
    train_f1_score = 2 * (train_precision * train_recall) / (train_precision + train_recall + 1e-12)

    return train_loss, train_acc, train_precision, train_recall, train_f1_score

def val_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module) -> Tuple[float, float]:

    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    val_loss, val_acc = 0, 0

    # Variables for precision, recall, and F1 score
    val_TP, val_FP, val_FN = 0, 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X, y #X.to(device), y.to(device)

            # 1. Forward pass
            val_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(val_pred_logits, y)
            val_loss += loss.item()

            # Calculate and accumulate accuracy
            val_pred_labels = val_pred_logits.argmax(dim=1)
            val_acc += ((val_pred_labels == y).sum().item()/len(val_pred_labels))

            # Update TP, FP, FN for precision, recall, and F1 score
            val_TP += ((val_pred_labels == y) & (y == 1)).sum().item()
            val_FP += ((val_pred_labels != y) & (y == 0)).sum().item()
            val_FN += ((val_pred_labels != y) & (y == 1)).sum().item()

    # Adjust metrics to get average loss and accuracy per batch
    val_loss = val_loss / len(dataloader)
    val_acc = val_acc / len(dataloader)

    # Calculate precision, recall, and F1 score
    val_precision = val_TP / (val_TP + val_FP + 1e-12)
    val_recall = val_TP / (val_TP + val_FN + 1e-12)
    val_f1_score = 2 * (val_precision * val_recall) / (val_precision + val_recall + 1e-12)

    return val_loss, val_acc, val_precision, val_recall, val_f1_score

C:\Users\sukri\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          val_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          result_csv_path: str = 'results.csv',
          model_save_path: str = 'model.pt') -> Dict[str, List]:

    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "train_precision": [],
               "train_recall": [],
               "train_f1_score": [],
               "val_loss": [],
               "val_acc": [],
               "val_precision": [],
               "val_recall": [],
               "val_f1_score": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc, train_precision, train_recall, train_f1_score = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer)
        val_loss, val_acc, val_precision, val_recall, val_f1_score = val_step(model=model,
                                        dataloader=val_dataloader,
                                        loss_fn=loss_fn)

        # Print out what's happening
        print(
            f"Epoch: {epoch + 1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"train_precision:{train_precision:.4f} |"
            f"train_recall:{train_recall:.4f} |"
            f"train_f1_score:{train_f1_score:.4f} |"
            f"val_loss: {val_loss:.4f} | "
            f"val_acc: {val_acc:.4f} |"
            f"val_precision:{val_precision:.4f} |"
            f"val_recall:{val_recall:.4f} |"
            f"val_f1_score:{val_f1_score:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["train_precision"].append(train_precision)
        results["train_recall"].append(train_recall)
        results["train_f1_score"].append(train_f1_score)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)
        results["val_precision"].append(val_precision)
        results["val_recall"].append(val_recall)
        results["val_f1_score"].append(val_f1_score)


    # Save results to CSV
    with open(result_csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Epoch", "Train Loss", "Train Acc","train_precision", "train_recall", "train_f1_score",   "Val Loss", "Val Acc", "val_precision","val_recall","val_f1_score" ])
        for epoch, train_loss, train_acc, val_loss, val_acc in zip(range(1, epochs + 1),
                                                                    results["train_loss"],
                                                                    results["train_acc"],
                                                                    results["train_precision"],
                                                                    results["train_recall"],
                                                                    results["train_f1_score"],
                                                                    results["val_loss"],
                                                                    results["val_acc"],
                                                                    results["val_precision"],
                                                                    results["val_recall"],
                                                                    results["val_f1_score"]):
            csvwriter.writerow([epoch, train_loss, train_acc, val_loss, val_acc])

    # Save model
    torch.save(model.state_dict(), model_save_path)
    torch.save(model, model_save_path)

    return results


In [11]:
# Create optimizer and loss function
optimizer = torch.optim.Adam(params=pretrained_vit_addtn.parameters(),
                              lr=0.0001,#,  # Adjust the learning rate
    weight_decay=0.0001)#lr=1e-3,
loss_fn = torch.nn.CrossEntropyLoss()
'''
early_stopping_callback = keras.callbacks.EarlyStopping(
          monitor='val_accuracy',
          patience= 10,  # Number of epochs with no improvement after which training will be stopped
          mode='max',  # 'max' if the monitored metric should be maximized, 'min' if minimized
          verbose=1
)
'''

# Train the classifier head of the pretrained ViT feature extractor model
#set_seeds()
pretrained_vit_results = train(model=pretrained_vit_addtn,
                                      train_dataloader=train_dataloader_pretrained,
                                      val_dataloader=val_dataloader_pretrained,
                                      optimizer=optimizer,
                                      loss_fn=loss_fn,
                                      epochs=10,
                                      result_csv_path='/python_programming/FYP_2022/viadnet/model/pretrained_vit_results_addtn_lsacprf.csv',
                                      model_save_path='/python_programming/FYP_2022/viadnet/model/viadnet_addtn_lsacprf.pt')#

 10%|███████▉                                                                       | 1/10 [27:15<4:05:17, 1635.27s/it]

Epoch: 1 | train_loss: 0.1431 | train_acc: 0.9680 | train_precision:0.9795 |train_recall:0.9560 |train_f1_score:0.9676 |val_loss: 0.1620 | val_acc: 0.9524 |val_precision:0.9380 |val_recall:0.9680 |val_f1_score:0.9528


 20%|███████████████▊                                                               | 2/10 [54:12<3:36:35, 1624.45s/it]

Epoch: 2 | train_loss: 0.0284 | train_acc: 0.9945 | train_precision:0.9930 |train_recall:0.9960 |train_f1_score:0.9945 |val_loss: 0.1284 | val_acc: 0.9544 |val_precision:0.9365 |val_recall:0.9740 |val_f1_score:0.9549


 30%|███████████████████████                                                      | 3/10 [1:14:42<2:48:30, 1444.29s/it]

...

 90%|████████████████████████████████████████████████████████████████████████        | 9/10 [2:06:17<09:51, 591.15s/it]

Epoch: 9 | train_loss: 0.0019 | train_acc: 1.0000 | train_precision:1.0000 |train_recall:1.0000 |train_f1_score:1.0000 |val_loss: 0.1107 | val_acc: 0.9633 |val_precision:0.9461 |val_recall:0.9820 |val_f1_score:0.9637


100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [2:14:59<00:00, 809.93s/it]

Epoch: 10 | train_loss: 0.0018 | train_acc: 1.0000 | train_precision:1.0000 |train_recall:1.0000 |train_f1_score:1.0000 |val_loss: 0.1393 | val_acc: 0.9593 |val_precision:0.9371 |val_recall:0.9840 |val_f1_score:0.9600


ValueError: ignored

Plot training data

In [ ]:
import pandas as pd

# Plot loss curves of a model
def plot_loss_curves(data):

    loss = data["Train Loss"].values
    val_loss = data["Val Loss"].values

    accuracy = data["Train Acc"].values
    val_accuracy = data["Val Acc"].values

    print('accuracy:', accuracy)
    print('val_accuracy:', val_accuracy)

    epochs = range(len(loss))

    '''
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()
    '''

In [ ]:
# Plot the loss curves
data = pd.read_csv('/python_programming/FYP_2022/viadnet/pretrained_vit_results0.csv')

plot_loss_curves(data)

accuracy: [0.99702381 0.99950397 1.         1.         1.         1.
 1.         1.         1.         1.        ]
val_accuracy: [0.94345238 0.94940476 0.9484127  0.95138889 0.95238095 0.95634921
 0.95734127 0.95833333 0.95734127 0.95734127]


Prediction on test dataset

In [ ]:
# Predict on a target image with a target model
import torch
import torchvision
from torchvision import transforms
#import matplotlib.pyplot as plt

from typing import List, Tuple

from PIL import Image

def pred_and_plot_image(
    model: torch.nn.Module,
    class_names: List[str],
    image_path: str,
    image_name: str,
    image_size: Tuple[int, int] = (224, 224),
    transform: torchvision.transforms = None
):


    # Open image
    img = Image.open(image_path)

    # Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose(
            [
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    ### Predict on image ###

    with torch.inference_mode():
        # Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = image_transform(img).unsqueeze(dim=0)

        # Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image)#.to(device))

    # Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)#torch.softmax(target_image_pred, dim=1)
    target_image_pred_probsval = target_image_pred_probs.max().item()

    # Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1).item()

    target_image_pred_classname = class_names[target_image_pred_label]

    #print('Pred:', class_names[target_image_pred_label], '|',  'Prob:', target_image_pred_probs.max():.3f)
    print('image name:', image_name, '|', 'Pred label:', target_image_pred_label, '|', 'pred prob:', round(target_image_pred_probsval,3), '|', 'class name:', target_image_pred_classname)

    #csvwriter.writerow(["Image_name", "Pred label", "Pred Prob"])
 # Save results to CSV
    with open('/python_programming/FYP_2022/viadnet/testresult_vit.csv', 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([image_name, target_image_pred_label, round(target_image_pred_probsval,3), class_name])


In [ ]:
test_image_path = "/python_programming/FYP_2022/billboard_dataset/test/Billboard/scene_1501.jpg"

#class_names = ['Billbord', 'No-billboard'] #0, 1

# Predict on test image
pred_and_plot_image(model=pretrained_vit,
                    image_path=test_image_path,
                    class_names=class_names)

test_image_path = "/python_programming/FYP_2022/billboard_dataset/test/No_Billboard/scene_1501.jpg"

#class_names = ['Billbord', 'No-billboard'] #0, 1

# Predict on test image
pred_and_plot_image(model=pretrained_vit,
                    image_path=test_image_path,
                    class_names=class_names)

Pred label: tensor([0]) | pred prob: tensor(0.9648)
Pred label: tensor([1]) | pred prob: tensor(0.9979)


In [ ]:

import os
def pred_on_directory(model, class_names, directory_path, file_extension=".jpg"):
    for filename in os.listdir(directory_path):
        if filename.endswith(file_extension):
            image_path = os.path.join(directory_path, filename)
            image_name = os.path.basename(image_path)
            #print('image name:', image_name)
            pred_and_plot_image(model, class_names, image_path, image_name)

# Directory path for test images
test_directory_path = "/python_programming/FYP_2022/billboard_dataset/test/Billboard"

# Predict on the whole test directory
pred_on_directory(model=pretrained_vit,
                  class_names=class_names,
                  directory_path=test_directory_path)


image name: scene_1501.jpg | Pred label: tensor([0]) | pred prob: tensor(0.9648)
image name: scene_1502.jpg | Pred label: tensor([0]) | pred prob: tensor(0.9169)
image name: scene_1503.jpg | Pred label: tensor([0]) | pred prob: tensor(0.9868)
image name: scene_1504.jpg | Pred label: tensor([0]) | pred prob: tensor(0.5799)
image name: scene_1505.jpg | Pred label: tensor([0]) | pred prob: tensor(0.9916)
image name: scene_1506.jpg | Pred label: tensor([0]) | pred prob: tensor(0.8660)
image name: scene_1507.jpg | Pred label: tensor([1]) | pred prob: tensor(0.6926)
image name: scene_1508.jpg | Pred label: tensor([0]) | pred prob: tensor(0.8000)
image name: scene_1509.jpg | Pred label: tensor([1]) | pred prob: tensor(0.9907)
image name: scene_1510.jpg | Pred label: tensor([0]) | pred prob: tensor(0.9908)
image name: scene_1511.jpg | Pred label: tensor([0]) | pred prob: tensor(0.9997)
image name: scene_1512.jpg | Pred label: tensor([0]) | pred prob: tensor(0.6167)
image name: scene_1513.jpg |

In [ ]:
import os

def pred_on_directory(model, class_names, directory_path, file_extension=".jpg"):
    for filename in os.listdir(directory_path):
        if filename.endswith(file_extension):
            image_path = os.path.join(directory_path, filename)
            image_name = os.path.basename(image_path)
            #print('image name:', image_name)
            pred_and_plot_image(model, class_names, image_path, image_name)

# Directory path for test images
test_directory_path = "/python_programming/FYP_2022/billboard_dataset/test/No_Billboard"

# Predict on the whole test directory
pred_on_directory(model=pretrained_vit,
                  class_names=class_names,
                  directory_path=test_directory_path)

image name: 0004.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0009.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0024.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0046.jpg | Pred label: tensor([1]) | pred prob: tensor(1.)
image name: 0068.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0075.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0083.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 111.jpg | Pred label: tensor([1]) | pred prob: tensor(0.9989)
image name: 114.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 115.jpg | Pred label: tensor([1]) | pred prob: tensor(0.9997)
image name: 12.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 13 (2).jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 130.jpg | Pred label: tensor([1]) | pred prob: tensor(0.8815)
image name: 14 (2).jpg | Pred lab

In [ ]:
def pred_on_directory(model, class_names, directory_path, file_extension=".jpg"):
    for filename in os.listdir(directory_path):
        if filename.endswith(file_extension):
            image_path = os.path.join(directory_path, filename)
            image_name = os.path.basename(image_path)
            #print('image name:', image_name)
            pred_and_plot_image(model, class_names, image_path, image_name)

# Directory path for test images
test_directory_path = "/python_programming/FYP_2022/billboard_dataset/test1"

# Predict on the whole test directory
pred_on_directory(model=pretrained_vit,
                  class_names=class_names,
                  directory_path=test_directory_path)

image name: 0004.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0009.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0024.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0046.jpg | Pred label: tensor([1]) | pred prob: tensor(1.)
image name: 0068.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0075.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 0083.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 111.jpg | Pred label: tensor([1]) | pred prob: tensor(0.9989)
image name: 114.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 115.jpg | Pred label: tensor([1]) | pred prob: tensor(0.9997)
image name: 12.jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 13 (2).jpg | Pred label: tensor([1]) | pred prob: tensor(1.0000)
image name: 130.jpg | Pred label: tensor([1]) | pred prob: tensor(0.8815)
image name: 14 (2).jpg | Pred lab